In [7]:
#!pip3 install -r ~/requirements.txt
!pip3 install pandas
!pip3 install scikit-learn

     |████████████████████████████████| 24.8 MB 26.9 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 106.4 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 105.8 MB/s eta 0:00:01


In [10]:
!pip3 install mlflow
import mlflow.spark

     |████████████████████████████████| 17.0 MB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 60.5 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 85.0 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 111.2 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 1.1 MB/s s eta 0:00:01
     |████████████████████████████████| 62 kB 264 kB/s eta 0:00:011
     |████████████████████████████████| 233 kB 105.2 MB/s eta 0:00:01
     |████████████████████████████████| 566 kB 102.7 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cml 1.0.0 requires requests==v2.27.1, but you have requests 2.28.2 which is incompatible.


ImportError: cannot import name 'ModelSignature' from 'mlflow.models' (/opt/cmladdons/python/site-packages/mlflow/models/__init__.py)

In [8]:
import os
import warnings
import sys
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

ImportError: cannot import name 'ModelSignature' from 'mlflow.models' (/opt/cmladdons/python/site-packages/mlflow/models/__init__.py)

In [10]:
import os
import warnings
import sys
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import logging
import json
import shutil
import datetime
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
import cml.data_v1 as cmldata

In [13]:
CONNECTION_NAME = "se-aw-mdl"

conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [14]:
training_df = spark.createDataFrame(
[
    ("0", "a b c d e spark", 1.0),
    ("1", "b d", 0.0),
    ("2", "spark f g h", 1.0),
    ("3", "hadoop mapreduce", 0.0),
],
["id", "text", "label"],
)

In [ ]:
#mlflow.end_run()

def exp1():



    mlflow.set_experiment("sparkml-experiment")

    ##EXPERIMENT 1

    training_df.writeTo("spark_catalog.default.training").using("iceberg").createOrReplace()
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    ### SHOW TABLE HISTORY AND SNAPSHOTS
    spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]

    ##EXPERIMENT 2

    ### ICEBERG INSERT DATA - APPEND FROM DATAFRAME

    # PRE-INSERT
    #spark.sql("SELECT * FROM spark_catalog.default.training").show()

    #temp_df = spark.sql("SELECT * FROM spark_catalog.default.training")
    #temp_df.writeTo("spark_catalog.default.training").append()
    #training_df = spark.sql("SELECT * FROM spark_catalog.default.training")

    # PROST-INSERT
    #spark.sql("SELECT * FROM spark_catalog.default.training").show()

    #spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    #spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    #snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    #committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    #parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]

    ##EXPERIMENT 3

    #Replace Snapshot ID here
    #snapshot_id = 2693814795059767550
    #training_df = spark.read.option("snapshot-id", snapshot_id).table("spark_catalog.default.training")

    #committed_at = spark.sql("SELECT committed_at FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).collect()[0][0].strftime('%m/%d/%Y')
    #parent_id = str(spark.sql("SELECT parent_id FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).tail(1)[0][0])

    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": training_df.count()
    }

    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=10
        regParam=0.001

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)

    mlflow.end_run()



    #spark.stop()
